<a href="https://colab.research.google.com/github/stavco9/datastreaming-final-project/blob/main/Data_Analysis_DeezyMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install DeezyMatch
!pip install thefuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 

In [ ]:
import os
import tqdm
import time
import pandas as pd
import numpy as np
from DeezyMatch import candidate_ranker, candidate_ranker_init
from thefuzz import process, fuzz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset"

Mounted at /content/drive
/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset


In [ ]:
words_set = []
for i in range(10):
  candidates_file = f"./candidates/dataset_candidates_{i}.txt"
  with open(candidates_file, 'r') as f:
    words_list = f.read().split('\n')
    words_set.append({"words_list": words_list})

In [ ]:
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, \
or a wartermelon if possibe. I hope i don't have an \
errof in Londaon and in Jerusealm"

In [52]:
def format_word(word):
  if len(word) > 1:
    last_char = word[-1]
    while not last_char.isalpha():
      word = word[:-1]
      last_char = word[-1]

    first_char = word[0]
    while not first_char.isalpha():
      word = word[1:]
      first_char = word[0]

    word = word.lower()

  return word

In [46]:
for index, words_list in enumerate(words_set):
    time_start_naive = time.time()
    words_set[index]["naive_matches"] = {}
    words_set[index]["naive_matches"]["origin_words"] = []
    words_set[index]["naive_matches"]["matched_words"] = []
    words_set[index]["naive_matches"]["match_scores"] = []
    for word in some_sentance_with_error.split():
      words_set[index]["naive_matches"]["origin_words"].append(format_word(word))
      word_match_results = [x for x in process.extract(word, words_list["words_list"], scorer=fuzz.ratio)]
      words_set[index]["naive_matches"]["matched_words"].append(format_word(word_match_results[0][0]) if len(word_match_results) > 0 else "None")
      words_set[index]["naive_matches"]["match_scores"].append(int(word_match_results[0][1]) / 100 if len(word_match_results) > 0 else 0)
    time_end_naive = time.time()
    words_set[index]["time_naive_sec"] = (time_end_naive - time_start_naive)

In [47]:
for index in range(10):
  # Ranking on-the-fly
  # find candidates from candidate_scenario
  # for queries specified by the `query` argument
  time_start_deezy = time.time()
  ranker = \
      candidate_ranker_init(candidate_scenario=f"./combined/candidates_test_{index}",
                      query=[x.replace("-", " ") for x in some_sentance_with_error.split()],
                      ranking_metric="faiss",
                      selection_threshold=10.,
                      num_candidates=1,
                      search_size=100,
                      output_path=f"ranker_results/test_candidates_deezymatch_on_the_fly_{index}",
                      pretrained_model_path="./models/wikigaz_en_model/wikigaz_en_model.model",
                      pretrained_vocab_path="./models/wikigaz_en_model/wikigaz_en_model.vocab",
                      number_test_rows=len(some_sentance_with_error))

  ranker.rank()
  words_set[index]["deezy_match_df"] = ranker.output
  time_end_deezy = time.time()
  words_set[index]["time_deezy_sec"] = (time_end_deezy - time_start_deezy)

2024-03-24 22:35:41 d36e1995d898 [INFO] read input file: ./combined/candidates_test_0/train_conf.yaml
2024-03-24 22:35:41 d36e1995d898 [INFO] pytorch will use: cuda
2024-03-24 22:35:41 d36e1995d898 [INFO] use a dataframe in test_tokenize.
2024-03-24 22:35:41 d36e1995d898 [INFO] number of labels, True: 0 and False: 30
2024-03-24 22:35:41 d36e1995d898 [INFO] skipping 0 lines


2024-03-24 22:35:41 d36e1995d898 [INFO] save test-data-class: tmp_c81b75d7-83d3-43be-8ef8-ef88cc54a351/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_c81b75d7-83d3-43be-8ef8-ef88cc54a351/query/embeddings/rnn_fwd*
0000000 tmp_c81b75d7-83d3-43be-8ef8-ef88cc54a351/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_c81b75d7-83d3-43be-8ef8-ef88cc54a351/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_c81b75d7-83d3-43be-8ef8-ef88cc54a351/query/embeddings/rnn_bwd*
0000000 tmp_c81b75d7-83d3-43be-8ef8-ef88cc54a351/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_c81b75d7-83d3-43be-8ef8-ef88cc54a351/query/embeddings/rnn_indxs_0
--- 2628.291318178177 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 5, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 30, searched: 100
=========== Start the search for 3 pleases
ID: 4/30

2024-03-24 22:35:44 d36e1995d898 [INFO] save test-data-class: tmp_1e6532fc-f0b2-425e-bdaa-bf4895756a5c/query/dataframe.df


...

  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_49a65af8-8043-47fb-b505-a9fe1117072e/query/embeddings/rnn_fwd*
0000000 tmp_49a65af8-8043-47fb-b505-a9fe1117072e/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_49a65af8-8043-47fb-b505-a9fe1117072e/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_49a65af8-8043-47fb-b505-a9fe1117072e/query/embeddings/rnn_bwd*
0000000 tmp_49a65af8-8043-47fb-b505-a9fe1117072e/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_49a65af8-8043-47fb-b505-a9fe1117072e/query/embeddings/rnn_indxs_0
--- 2644.906648874283 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 4, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 29, searched: 100
=========== Start the search for 3 pleases
ID: 4/30

2024-03-24 22:36:00 d36e1995d898 [INFO] save test-data-class: tmp_c2b3e2ba-68e7-4828-b7bf-2b6a0bf1bc18/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_c2b3e2ba-68e7-4828-b7bf-2b6a0bf1bc18/query/embeddings/rnn_fwd*
0000000 tmp_c2b3e2ba-68e7-4828-b7bf-2b6a0bf1bc18/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_c2b3e2ba-68e7-4828-b7bf-2b6a0bf1bc18/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_c2b3e2ba-68e7-4828-b7bf-2b6a0bf1bc18/query/embeddings/rnn_bwd*
0000000 tmp_c2b3e2ba-68e7-4828-b7bf-2b6a0bf1bc18/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_c2b3e2ba-68e7-4828-b7bf-2b6a0bf1bc18/query/embeddings/rnn_indxs_0
--- 2646.784895181656 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/30 -- Number of found candidates so far: 2, searched: 100
=========== Start the search for 1 wanna
ID: 2/30 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 2 appel
ID: 3/30 -- Number of found candidates so far: 33, searched: 100
=========== Start the search for 3 pleases
ID: 4/30

In [54]:
for index in range(10):
    deezy_match_df = words_set[index]["deezy_match_df"]
    words_set[index]["deezy_matches"] = {}
    words_set[index]["deezy_matches"]["origin_words"] = []
    words_set[index]["deezy_matches"]["matched_words"] = []
    words_set[index]["deezy_matches"]["match_distance"] = []
    for word in some_sentance_with_error.split():
      word_df_row = deezy_match_df.loc[deezy_match_df['query'] == word].iloc[0]
      words_set[index]["deezy_matches"]["origin_words"].append(format_word(word))
      words_set[index]["deezy_matches"]["matched_words"].append(format_word(list(word_df_row['faiss_distance'].keys())[0]) if bool(word_df_row['faiss_distance']) else None)
      words_set[index]["deezy_matches"]["match_distance"].append(list(word_df_row['faiss_distance'].values())[0] if bool(word_df_row['faiss_distance']) else -1)

In [55]:
for index, words_list in enumerate(words_set):
   print(f"Total time of dataset {index + 1} is {words_list['time_naive_sec']}s")
   print(f"Origin words for dataset {index + 1} are {words_list['naive_matches']['origin_words']}")
   print(f"Matches words for dataset {index + 1} are {words_list['naive_matches']['matched_words']}")
   print(f"Matches scores for dataset {index + 1} are {words_list['naive_matches']['match_scores']}")

Total time of dataset 1 is 3.9515442848205566s
Origin words for dataset 1 are ['i', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone', 'or', 'a', 'wartermelon', 'if', 'possibe', 'i', 'hope', 'i', "don't", 'have', 'an', 'errof', 'in', 'londaon', 'and', 'in', 'jerusealm']
Matches words for dataset 1 are ['i', 'wanna', 'appeal', 'please', 'or', 'a', 'bananas', 'with', 'a', 'paper', 'and', 'a', 'lemon', 'or', 'a', 'waterloo', 'if', 'possible', 'i', 'hope', 'i', "don't", 'have', 'an', 'error', 'in', 'london', 'and', 'in', 'jerusalem']
Matches scores for dataset 1 are [1.0, 1.0, 0.91, 0.92, 1.0, 1.0, 0.86, 1.0, 1.0, 0.89, 1.0, 1.0, 0.91, 1.0, 1.0, 0.74, 1.0, 0.93, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 1.0, 0.92, 1.0, 1.0, 0.89]
Total time of dataset 2 is 3.9883639812469482s
Origin words for dataset 2 are ['i', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone', 'or', 'a', 'wartermelon', 'if', 'possibe', 'i', 'h

In [56]:
for index in range(10):
   print(f"Total time of dataset {index + 1} is {words_set[index]['time_deezy_sec']}s")
   print(f"Origin words for dataset {index + 1} are {words_set[index]['deezy_matches']['origin_words']}")
   print(f"Matches words for dataset {index + 1} are {words_set[index]['deezy_matches']['matched_words']}")
   print(f"Matches scores for dataset {index + 1} are {words_set[index]['deezy_matches']['match_distance']}")

Total time of dataset 1 is 2.081695318222046s
Origin words for dataset 1 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone', 'or', 'a', 'wartermelon', 'if', 'possibe', 'I', 'hope', 'i', "don't", 'have', 'an', 'errof', 'in', 'londaon', 'and', 'in', 'jerusealm']
Matches words for dataset 1 are ['I', 'wanna', 'papel', 'settles', 'or', 'a', 'bandanna', 'with', 'a', 'parker', 'and', 'a', 'lemon', 'or', 'a', 'trawled', 'if', 'possible', 'I', 'hope', 'I', "don't", 'have', 'an', 'error', 'in', 'london', 'and', 'in', 'jerusalem']
Matches scores for dataset 1 are [0.0, 0.0, 4.7146, 5.0393, 0.0, 0.0, 3.2076, 0.0, 0.0, 3.4475, 0.0, 0.0, 6.6457, 0.0, 0.0, 7.4863, 0.0, 4.5969, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.2706, 0.0, 3.8871, 0.0, 0.0, 5.5064]
Total time of dataset 2 is 2.4309515953063965s
Origin words for dataset 2 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone', 'or', 'a', 'wartermelon', 'if

In [57]:
correct_sentence = "I wanna apple please or a banana with a paer and a lemon, \
or a watermelon if possible. I hope i don't have an \
error in London and in Jerusalem"

In [58]:
set_corrections = {}
for index, words_list in enumerate(words_set):
  set_corrections[f"naive_{index}"] = words_list['naive_matches']['matched_words']
  set_corrections[f"deezy_{index}"] = words_list['deezy_matches']['matched_words']

In [59]:
df_comparisons = pd.DataFrame(data=set_corrections, index=[format_word(x) for x in correct_sentence.split()])
df_comparisons

,naive_0,deezy_0,naive_1,deezy_1,naive_2,deezy_2,naive_3,deezy_3,naive_4,deezy_4,naive_5,deezy_5,naive_6,deezy_6,naive_7,deezy_7,naive_8,deezy_8,naive_9,deezy_9
I,i,I,i,I,i,i,i,ⓘ,i,I,i,I,i,I,i,i,i,i,i,I
wanna,wanna,wanna,wanna,wanna,wanna,wanna,wanna,wanna,anna,with,anna,ioanna,anna,hanna,wanna,wanna,anna,hanna,anna,think
apple,appeal,papel,appeal,appealed,appeal,appice,appel,appel,appeal,appealed,appeal,appealed,appeal,appealed,appeal,appealed,appeal,apoel,appeal,padel
please,please,settles,leases,stallion's,please,safeties,please,please,please,settlers,please,settles,please,peeresses,please,lipstick,please,settles,please,settles
or,or,or,or,or,or,or,or,or,or,or,or,or,or,or,or,or,or,or,or,or
a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a
banana,bananas,bandanna,banana,banana,banana,banana,annan,nana,banana,banana,banana,banana,banana,banana,lannan,anandamath,banana,nana,bandanna,bandanna
with,with,with,with,with,with,with,with,with,with,with,with,with,with,with,with,with,with,with,with,with
a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a,a
paer,paper,parker,paper,parker,paper,parker,paper,parker,paper,parker,paper,parker,paper,parker,paper,parker,paper,parker,paper,parker
